In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.cassandra._
val sparkSession = SparkSession.builder().config(sc.getConf).getOrCreate()
import sparkSession.implicits._

In [2]:
sparkSession.read.cassandraFormat(keyspace = "yelp", table = "review").load().createOrReplaceTempView("review")
sparkSession.read.cassandraFormat(keyspace = "yelp", table = "business").load().createOrReplaceTempView("business")

In [1]:
sparkSession.sql("""
    select
        business_id,
        name business_name,
        bad_thing_count,
        review_id review_happened_at,
        date
    from(
        select
            review_id,
            business_id,
            name,
            date,
            bad_thing_count,
            case
                when
                     stars<=3 and
                     bad_thing_count>=3
                 then 1
                 else 0
                 end
                 as bad_thing_happened        
        from(
            select
                review_id,
                business_id,
                name,
                stars,
                date,
                sum(bad_thing_lasting) over 
                         (PARTITION BY business_id ORDER BY date ROWS BETWEEN 5 PRECEDING AND CURRENT ROW) bad_thing_count
            from(
                select
                    r.review_id,
                    b.business_id,
                    b.name,
                    r.stars,
                    r.date,
                    case
                        when
                            lag(r.stars) over (PARTITION BY b.business_id ORDER BY r.date)<=3
                        then 1 
                        else 0 
                        end
                        as bad_thing_lasting
                from business b
                    inner join review r on r.business_id = b.business_id
            ) T
        )
    )
    where bad_thing_happened=1
    order by date
"""
).show(50,false)

+----------------------+----------------------------------+---------------+----------------------+----------+
|business_id           |business_name                     |bad_thing_count|review_happened_at    |date      |
+----------------------+----------------------------------+---------------+----------------------+----------+
|6TKJDgTrUoD3zu77A4JHnQ|Coco's Family Restaurant          |3              |aNuInnV1XZ1ZCWRSnSCgCA|2007-09-22|
|st_pnALlGB6OCWkb4e14mg|Bada Boom Pasta Room              |3              |TOQVb5aFxBEzMNTq0v5euQ|2008-04-20|
|st_pnALlGB6OCWkb4e14mg|Bada Boom Pasta Room              |3              |wX_B_xiQbUxLy_S9zrg-og|2008-06-30|
|Ja4stXdNYr39u5CZHMNtjw|Cien Agaves Tacos & Tequila       |3              |IMEGRo_3_ClqsNHw2jHsyA|2008-10-17|
|saWZO6hB4B8P-mIzS1--Xw|Kabob Palace                      |3              |Mxunr5MHhLGRK-yP1VFtqw|2008-12-28|
|Gi48Z9ZPlEUgQU1xo_7Hjw|Espresso Royale                   |3              |7Z1zarmPE3CrKvR3SOuVtg|2009-01-18|
|Ja4stXdNY